In [1]:
import pandas as pd
file_path = r"C:\Users\141823\OneDrive - Etex Group\Desktop\Etex AU - Finance - Finance\19.Business Systems\Metal Report\BW Original Data.xlsx"
mapping_path=r"C:\Users\141823\OneDrive - Etex Group\Desktop\Etex AU - Finance - Finance\19.Business Systems\Metal Report\Mapping.xlsx"
# Load Excel file
xls = pd.ExcelFile(file_path)
xls1= pd.ExcelFile(mapping_path)

# Read "Raw Data" and "Region Mapping" sheets
df23=pd.read_excel(xls, sheet_name="2023", header=0)
df24=pd.read_excel(xls, sheet_name="2024", header=0)
df25=pd.read_excel(xls, sheet_name="2025", header=0)

#read mapping data
sales_office_mapping=pd.read_excel(xls1, sheet_name="Sales Office", header=0)
region_mapping=pd.read_excel(xls1, sheet_name="Region Mapping", header=0)
group_mapping=pd.read_excel(xls1, sheet_name="Grouping", header=0)
df_budget=pd.read_excel(xls1, sheet_name="2025 Budget", header=0)
# Combine df23, df24, and df25
df = pd.concat([df23, df24, df25], ignore_index=True)



In [2]:
#-------------------------------------------Product Masterdata----------------------------
import pandas as pd
from openpyxl import load_workbook



# Read both material and customer master data
material_df = pd.read_excel(r"C:\Users\141823\OneDrive - Etex Group\Desktop\Etex AU - Finance - Finance\00. Master data\Material Master Data _Etex AU 5500.xlsx", sheet_name=0, header=None)
customer_df = pd.read_excel(r"C:\Users\141823\OneDrive - Etex Group\Desktop\Etex AU - Finance - Finance\00. Master data\Customer Master Data _Etex.XLSX", sheet_name='Customer Masterdata')

# Set the second row (index 1) as the column headers
material_df.columns = material_df.iloc[1]  # Set second row as header
material_df = material_df.iloc[2:].reset_index(drop=True)  # Drop first two rows

# Pick needed columns and remove duplicates
material_subset = material_df[['Material', 'Board / Metal','P&L Classification','Group 1 Category','BRAND - Siniat / Promat / Innova / Equitone','Profile Text','Item Group']].rename(
    columns={'BRAND - Siniat / Promat / Innova / Equitone': 'Brand',
             'P&L Classification':'P&L Category',
             'Group 1 Category':'Manual Classification'})
# material_subset = material_subset.drop_duplicates(subset=['Material', 'Brand'])

# Merge with Raw Data
df = df.merge(material_subset, on='Material', how='left')

# Filter only for 'SINIAT' and 'GTEK' brands
df = df[df['Brand'].isin(['SINIAT', 'GTEK'])]

#Filter only for 'Metal'
df = df[df['Board / Metal'] == 'METAL']


In [3]:
#------------------------------------Customer Masterdata-------------------------------
# Select columns by index
customer_subset = customer_df.iloc[:, [0, 4, 9]]

# Rename columns
customer_subset.columns = ['Customer','Sales Office', 'Customer Name Master']
customer_subset=customer_subset.drop_duplicates(subset=['Customer', 'Customer Name Master'])

#Merge with df
df=df.merge(customer_subset, on='Customer', how='left')

# Replace 'Customer Name' in df with 'Customer Name Master' and rename it back to 'Customer Name'
df['Customer Name'] = df['Customer Name Master']

# Drop the extra 'Customer Name Master' column
df = df.drop(columns=['Customer Name Master'])

In [4]:
#--------------------------------Sales Office-------------
#Merge with Region Mapping to replace "Region" with "BW Region"
df = df.merge(sales_office_mapping, on='Sales Office', how='left')
region_mapping=region_mapping[['Region','BW Region']]
df=df.merge(region_mapping, on='Region', how='left')
# Replace "Region" with "BW Region", keeping original values if no match
df['Region'] = df['Sales Office Final'].fillna(df['BW Region'])

#Drop BW Region now, drop 'Brand' and 'Board / Metal'
df = df.drop(columns=['Sales Office Final','Brand','Board / Metal', 'BW Region'])
# Calculate Margin
df['Margin'] = (
    df['Actual Cost Of Goods Sold'].fillna(0) +
    df['Gross Sales (Invoice)'].fillna(0) +
    df['Provisions'].fillna(0)
)

# Calculate Margin after Transport
df['Margin after Transport'] = (
    df['Margin'] +
    df['Transport Surcharge'].fillna(0) +
    df['Transport to Customers'].fillna(0) +
    df['Interplant Transport'].fillna(0)
)

In [5]:
#--------------------Save df to final file------------------
# Load the workbook
file_path=r"C:\Users\141823\OneDrive - Etex Group\Desktop\Etex AU - Finance - Finance\19.Business Systems\Metal Report\Metal Report-Raw Data.xlsx"
wb = load_workbook(file_path)
ws = wb["Raw Data"]

#file_path=r"C:\Users\139491\OneDrive - Etex Group\Finance\06. Revenue & Pricing\Etex\19.Business Systems\Metal Report\Metal Report-Raw Data.xlsx"
#wb = load_workbook(file_path)
#ws = wb["Raw Data"]
# Clear old data but keep headers
ws.delete_rows(2, ws.max_row)  # Deletes from row 2 downwards (keeps headers)

# Write updated DataFrame back to "Raw Data" (including headers)
for c_idx, column in enumerate(df.columns, start=1):
    ws.cell(row=1, column=c_idx, value=column)  # Write column headers

for r_idx, row in enumerate(df.itertuples(index=False), start=2):  # Start from row 2 (below headers)
    for c_idx, value in enumerate(row, start=1):
        ws.cell(row=r_idx, column=c_idx, value=value)

# Auto-adjust column widths
for col in ws.columns:
    max_length = 0
    col_letter = col[0].column_letter  # Get column letter (A, B, C, etc.)
    for cell in col:
        try:
            if cell.value:
                max_length = max(max_length, len(str(cell.value)))
        except:
            pass
    ws.column_dimensions[col_letter].width = max_length + 2  # Add padding

# Save the workbook
wb.save(file_path)
